# Obtain edges from a Pajek-like file

In [1]:
import time
import pandas as pd
import numpy as np
from nodes_to_pajek import *

In [2]:
s=time.time()
imax=100#000
for i in range(imax):
    for j in range(imax):
        a=1+1

print(time.time()-s)        

0.0011782646179199219


In [3]:
import pandas as pd

##  Obtain full nodes file.  Small example

In [4]:
df=pd.read_json('nodes_small.json')

In [5]:
df[:1]

,abstract,authors,citations,co-authors,creation_date,free_keywords,list_rcid,recid,references,standardized_keywords,title
0,The exploitation of unrenormalized canonical f...,"[Brandt, Richard A.]",[18551],"[Ng, Wing-Chiu, Young, Kenneth]",1976-05,[],"[3560, 3560, 3560, 3560, 3560, 3560, 3560, 356...",3560,"[87072, 1283, 86276, 1577, 95084, 54444, 99663...","[GAUGE FIELD THEORY: NONABELIAN, GAUGE FIELD T...",Canonical Methods in Nonabelian Gauge Theories


In [6]:
pj=nodes_to_pajek(df)

In [7]:
pj.shape

(4915, 2)

##  Main code to find the edges

In [ ]:
start_time=time.time()

DEBUG=True
if DEBUG: 
    #===log===
    f=open('kk.log','w')
    f.write('')
    f.close()
    #==DEBUG===========

id_col='recid'; refs_col='references'
# Half of the work is order by refs_col, and id_col
pj=pj.sort_values([refs_col,id_col]).reset_index(drop=True)
ed={}
t=()
new_ref=0
i=0
iprint=1000

while i<=pj.index[-1]:
    if not i%iprint:
        print(i,end='\r')
        if DEBUG:
            #===log===
            f=open('kk.log','a')
            f.write('{}\n'.format(i))
            f.close()
            #=======
    e1=pj.loc[i,id_col]
    #check next entry and beyond:
    for j in range(i+1,pj.index[-1]+1):
        #print('i,j',i,j) #,end='\r')
        e2=pj.loc[j,id_col]
        #DEBUG
        #if (e1,e2)==(190836,1644265):
        #    print('i,j,check',i,j)
        edges=False
        if pj.loc[i,refs_col]==pj.loc[j,refs_col]:
            edges=True
        #else:
            #print('newi',i,j)
            
        if edges:
            if e1<e2:
                t=(e1,e2)
                if not ed.get(t):
                    ed[t]=1
                    #DEBUG
                    #ed[t]=str( (i,j) )
                else: #increase the number of common refs_col
                    ed[t]=ed[t]+1
                    #DEBUG: 
                    #ed[t]=ed[t]+str(  (i,j)  )
        else:
            # Go to next i when nor additional edges are found
            break
    #search after j. 
    i=i+1 # Be sure to increase i to avoid infinite loop
    #if i>=25:
    #    break
    continue

print(time.time()-start_time)    

In [44]:
1052/60

17.533333333333335

In [47]:
N=lambda x:  x*1052/100000/60
npt=1000000
print( N(npt),'m') 
print( N(npt)/60,'h') 

175.33333333333334 m
2.9222222222222225 h


In [48]:
le=[{t:ed[t]} for t in ed.keys() if ed[t]>4 ]
len(le)

3018

In [49]:
edf=pd.DataFrame({'tmp':le})
edf['edges'] =edf.tmp.map(lambda x: list(x.keys())[0]  )
edf['len_ed']=edf.tmp.map(lambda x: list(x.values())[0]  )
edf=edf.drop('tmp',axis='columns')
edf[:3]

,edges,len_ed
0,"(4380, 666911)",5
1,"(15239, 1387684)",5
2,"(98947, 129333)",5


In [50]:
#edf.to_json('example1E5.json')

# Real case scenario
Full pajek file

In [10]:
df_all=pd.read_json('hep_records.json',lines=True)

In [11]:
dfr=df_all[df_all.references.map(len)>0].reset_index(drop=True)
dfr.shape

(896637, 10)

In [12]:
pjt=nodes_to_pajek(dfr)

In [13]:
pjt.shape

(21733815, 2)

In [51]:
pj=pjt.reset_index(drop=True)

In [52]:
pj.shape

(21733815, 2)

In [53]:
pj[:3]

,recid,references
0,76215,1
1,81651,1
2,128250,1


In [17]:
pj[:3]

,recid,references
0,76215,1
1,81651,1
2,128250,1


Go to the previous code and run there. 

## Backup

IMPORTANT COMMITS: 
* 44fa444ba3cc1f9a052bf80051f44a0d11c5838a
  * Here the counting of the slow and fast to grouping mathc
